LangChainMemory 
```
이전 대화 내용을 기억해서 문맥을 유지하는 역활 LangChain 0.3X부터는 LCEL 기반으로 체인을 구성, RunnableWithMessageHistory, ChatMessageHistory 등의 컴포넌트를 활용해서 세션별 대화 기록을 관리, 대화가 장기화될 경우 요약 메모리를 도입해서 과거 대화를 LLM으로 요약하고 축약된 형태로 저장해서 프롬프트의 길이문제를 해결
```

In [8]:
%pip install --quiet langchain langchain-openai python-dotenv langchain_redis

Note: you may need to restart the kernel to use updated packages.


In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
from langchain_core.chat_history import  InMemoryChatMessageHistory
# 메모리 객체 생성
history =  InMemoryChatMessageHistory()
history.add_user_message("안녕하세요 제 이름은 홍길동 입니다.")
history.add_ai_message('안녕하세요 홍길동님, 무엇을 도와드릴까요?')
# 현재까지의 대화 내용 확인
for msg in history.messages:
    print(f'{msg.type} : {msg.content}')

human : 안녕하세요 제 이름은 홍길동 입니다.
ai : 안녕하세요 홍길동님, 무엇을 도와드릴까요?


In [ ]:
# Radis 기반 채팅 기록 저장소
from langchain_redis import RedisChatMessageHistory
import os
REDIS_URL = os.getenv("REDIS_URL", "redis://localhost:6379")
session_id = 'user_123'
history =  RedisChatMessageHistory(session_id=session_id, redis_url=REDIS_URL)
history.add_user_message("안녕하세요 제 이름은 홍길동 입니다.")
history.add_ai_message('안녕하세요 홍길동님, 무엇을 도와드릴까요?')
# 현재까지의 대화 내용 확인
for msg in history.messages:
    print(f'{msg.type} : {msg.content}')

InMemoryChatMessageHistory(messages=[HumanMessage(content='안녕하세요 제 이름은 홍길동 입니다.', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요 홍길동님, 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={})])

In [9]:
# 세션기반 다중사용자 메모리 구조 구현 - 다중사용자 챗봇
# 핵심 :session_id를 키로 하는 메모리 저장소만들고 사용자의 대화는 키별로 저장한다.
from langchain_core.prompts import  ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
# 프롬프트
prompt = ChatPromptTemplate.from_messages([
    ("system",'당신은 뛰어난 한국어 상담 챗봇입니다 질문에 친절하고 자세히 답변해주세요'),
    # history키로 전달된 메세지 목록은 체인 실행시 해당 위치에 넣겠다는 의미
    MessagesPlaceholder(variable_name='history'), 
    ('human','{input}')
])
llm = ChatOpenAI(model='GPT-4o-mini',temperature=0)

In [10]:
# LCEL
from langchain_core.output_parsers import StrOutputParser
chain = prompt | llm | StrOutputParser()